In [1]:
list("Hello world".encode("utf-8"))

[72, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]

In [20]:
from typing import List,Dict

In [49]:
text = "aaabdaaabac"
#We need to find the most common byte pair, a byte here is a character ofc 
#O(n^2) brute force algo is check for every pair and keep count of freq 
tokens = text.encode("utf-8")
tokens = list(map(int,tokens))
print(tokens)

def get_stats(tokens):
    freq = {} 
    for pair in zip(tokens,tokens[1:]):
        freq[pair] = freq.get(pair,0)+1
    return freq

counts = get_stats(tokens=tokens)
ls = sorted(((v,k) for k,v in counts.items()),reverse=True)

[97, 97, 97, 98, 100, 97, 97, 97, 98, 97, 99]


In [51]:
print(ls[0][1])

(97, 97)


In [44]:
#Replacement strategy 
#Search for the pairs, when found, replace with some new token 
def transform(tokens, pair, new_token):
    ls = [] 
    i=0
    while i < len(tokens):
        if i < len(tokens)-1 and tokens[i]==pair[0] and tokens[i+1]==pair[1]:
            ls.append(new_token)
            i+=2 
        else:
            ls.append(tokens[i]) 
            i+=1
    return ls 

In [45]:
ls = transform(tokens=tokens, pair = (97,97), new_token= 999)
print(ls)

[999, 97, 98, 100, 999, 97, 98, 97, 99]


In [73]:
'''
    Now we need to go on until the string becomes of some desired vocab size say 276 
'''

def parseText(text:str)->List[int]:
    tokens = text.encode("utf-8")
    return list(map(int,tokens))

def get_stats(tokens):
    freq = {} 
    for pair in zip(tokens,tokens[1:]):
        freq[pair] = freq.get(pair,0)+1
    return freq

def transform(tokens, pair, new_token):
    ls = [] 
    i=0
    while i < len(tokens):
        if i < len(tokens)-1 and tokens[i]==pair[0] and tokens[i+1]==pair[1]:
            ls.append(new_token)
            i+=2 
        else:
            ls.append(tokens[i]) 
            i+=1
    return ls 

vocab_size = 260

''' 
aaabdaaabac
ZabdZabac 
ZXdZXac 
YdYac 
'''

tokens = parseText(text) #in range 0...255
num_merges = vocab_size - 256
ids = list(tokens) 
merges = {} 
for i in range(num_merges-1):
    stats = get_stats(ids)
    pair = ()
    if(len(stats)>0):
        pair = sorted(((v,k) for k,v in stats.items()),reverse=True)[0][1]

    idx = i+256 
    ids = transform(ids, pair, idx)
    merges[pair] = idx 





[257, 97, 98, 100, 257, 97, 98, 97, 99]
[97, 97, 97, 98, 100, 97, 97, 97, 98, 97, 99]
[256, 97, 98, 100, 256, 97, 98, 97, 99]
[257, 98, 100, 257, 98, 97, 99]
[258, 100, 258, 97, 99]
aaabdaaabac
[258, 100, 258, 97, 99]


In [65]:
vocab = {idx : bytes(idx) for idx in range(256)} 
print(vocab)

{0: b'', 1: b'\x00', 2: b'\x00\x00', 3: b'\x00\x00\x00', 4: b'\x00\x00\x00\x00', 5: b'\x00\x00\x00\x00\x00', 6: b'\x00\x00\x00\x00\x00\x00', 7: b'\x00\x00\x00\x00\x00\x00\x00', 8: b'\x00\x00\x00\x00\x00\x00\x00\x00', 9: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00', 10: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 11: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 12: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 13: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 14: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 15: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 16: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 17: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 18: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 19: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 20: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [76]:
print(merges)
print(ids)
ls = list(merges)

{(97, 97): 256, (256, 97): 257, (257, 98): 258}
[258, 100, 258, 97, 99]


In [78]:
rev_merges = {}
for k,v in merges.items(): 
    rev_merges[v] = k 



In [79]:
print(rev_merges)
#unroll 258 = 257:97,97,97, 98 dfs

{256: (97, 97), 257: (256, 97), 258: (257, 98)}


In [84]:
def split(token):
    global rev_merges
    ans = []
    if token > 255: 
        _token = rev_merges.get(token)
        for v in _token: 
                ans += split(v)
    else:
        ans.append(token)       
    return ans
        


In [85]:
split(258)

(257, 98)
(256, 97)
(97, 97)


[97, 97, 97, 98]

In [86]:
def decode(ids):
    #Dumb hack , since we know everything above 255 is a combined pair , we split 
    ls = []
    for token in ids: 
        ls += split(token)
    return ls 
    

In [87]:
ls = decode(ids)

(257, 98)
(256, 97)
(97, 97)
(257, 98)
(256, 97)
(97, 97)


In [88]:
print(ls)

[97, 97, 97, 98, 100, 97, 97, 97, 98, 97, 99]


In [89]:
bytecode = [vocab.get(idx) for idx in ls]

In [92]:
chr(vocab[97]) 

TypeError: an integer is required (got type bytes)